# 解析单个大文件
使用RecursiveCharacterTextSplitter 切分文档；使用 load_summarize_chain 创建总结链，总结链调用分割后的文档列表，输出文档总结； 使用 DashScopeEmbeddings 将切分的文本转换成向量，并存入FAISS向量数据库，使用RetrievalQA创建检索链

In [1]:
from langchain.chat_models import init_chat_model
from langchain_community.embeddings import DashScopeEmbeddings
import os
from dotenv import load_dotenv


load_dotenv()
# 初始化模型
os.environ["OPENAI_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")
os.environ["OPENAI_BASE_URL"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"

# 初始化LLM
llm = init_chat_model(model="qwen-plus", model_provider="openai")

# 初始化嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1",
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 读取文档
file_path = "./docs/state_of_the_union.txt"
loader = TextLoader(file_path, encoding='utf-8')
text = loader.load()
    
# 拆分大文本，防止超长输入
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # 每块约1000字符，可调整
    chunk_overlap=200,  # 两端的重叠部分长度
    length_function=len,
)
split_docs = text_splitter.split_documents(text)
print("step end.")

step end.


In [3]:
from langchain.chains.summarize import load_summarize_chain

# 创建总结链
summarychain = load_summarize_chain(llm, chain_type="map_reduce")

# 生成总结
summary = summarychain.invoke(split_docs)
print(summary["output_text"])
print("step end.")

D:\Code\full-stack-fastapi-template\backend\venv\Lib\site-packages\langchain_openai\chat_models\base.py:401: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


《人类简史》是以色列历史学家尤瓦尔·赫拉利所著、林俊宏翻译的畅销书，由中信出版社于2014年出版。全书从宏观视角回顾人类历史，分为认知革命、农业革命、人类融合与科学革命四部分，融合历史、生物学、物理学等多学科知识，反思人类中心主义、资本主义、科学滥用及幸福本质。书中指出，智人并非唯一人类物种，约1万年前才成为唯一幸存者，并迅速改变生态与社会结构。关于智人起源与尼安德特人互动，存在混种繁衍与替代理论，基因证据支持混种理论，但争议尚存。该书因其跨学科视角与深刻哲思受到广泛关注。
step end.


# chain_type常见种类：

### 1. "stuff"
- 原理：把检索到的所有文档内容直接“塞进”（stuff）提示词里，交给 LLM 处理。
- 优点：简单、直接，效果好（上下文不会被切割）。
- 缺点：如果检索结果太多，容易超过模型的 max_tokens。
- 👉 适合 小规模文档，比如 FAQ、配置文件。

### 2. "map_reduce"
- 原理：
  map：LLM 先对每个文档单独回答问题。
  reduce：再把这些回答合并，总结出一个最终回答。
- 优点：支持大量文档（分布式处理）。
- 缺点：回答可能丢失细节，因为 reduce 依赖 LLM 总结。
- 👉 适合 大规模文档，需要归纳总结的场景。

### 3. "refine"
- 原理：
  先用第一篇文档生成一个初步回答。
  然后依次遍历后续文档，每次都“改进 / refine”这个回答。
- 优点：逐步优化，能结合更多文档信息。
- 缺点：速度慢，可能遗忘前面的信息。
- 👉 适合 需要整合多个文档，且对准确性要求高的场景。

### 4. "map_rerank"
- 原理：
  map：对每个文档单独生成一个回答，并给出“相关性评分”。
  rerank：选择最相关的回答返回。
- 优点：能过滤掉噪声文档，提升回答质量。
- 缺点：可能漏掉一些辅助信息。
- 👉 适合 对相关性要求高的问答（例如法律文档检索）。

In [4]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
# 创建向量数据库
vector_store = FAISS.from_documents(split_docs, embeddings)
# 创建检索QA链
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

# 回答问题
result = qa_chain.invoke({"query": "什么东西可以体现认类与其他生物之间的不同"})
print(result["result"])
print("step end.")

根据上述内容，人类与其他生物之间的不同，可以从以下几个关键方面体现出来：

1. **强大的社交技巧和复杂的社会结构**  
   人类之所以具有突出的社交技巧，是因为早期的人类母亲在养育孩子时需要其他家族成员和邻居的持续协助。因此，演化更倾向于那些能够形成强大社会关系的种族。这种高度的社会合作使得人类能够建立复杂的社会结构。

2. **高度的可塑性和社会化能力**  
   人类出生时身体和大脑都尚未完全发育，这赋予了人类极高的可塑性。与大多数哺乳动物（出生时几乎“固定成型”）不同，人类可以被教育和社会化深刻地塑造。这种特性使得人类能够接受不同的文化、信仰和价值观，从而形成多样化的社会群体。

3. **大脑容量和认知能力**  
   人类的大脑明显大于其他动物。例如，现代智人的脑容量平均为1200~1400立方厘米，而其他哺乳动物的脑容量则小得多。这种大脑的进化使得人类具备了使用工具、学习、解决问题以及创造文化的能力。

4. **文化的发展和历史的积累**  
   人类在大约7万年前经历了“认知革命”，开始创造复杂的社会文化。这些文化随着时间的推移不断发展，最终形成了历史学、艺术、宗教、哲学等独特的人类领域。

5. **改变环境的能力**  
   虽然早期人类在食物链中处于中游位置，但随着时间的推移，人类逐渐掌握了狩猎大型动物、农业生产和科学技术，最终成为地球上最具影响力的物种。这种改变环境的能力是其他生物所不具备的。

6. **独特的合作和沟通能力**  
   人类能够通过语言和符号系统进行复杂的沟通，并基于共同的信念和规则进行大规模合作。这种能力使得人类能够超越家庭或小群体的限制，形成国家、宗教、经济体系等更大规模的社会组织。

综上所述，人类与其他生物的不同体现在其高度的社会性、强大的可塑性、发达的大脑和认知能力、复杂的文化发展，以及改变环境和大规模合作的能力。
